# Register CSV Data With Athena
This will create an Athena table in the Glue Catalog (Hive Metastore).

Now that we have a database, we’re ready to create a table that’s based on the `Amazon Customer Reviews Dataset`. We define the columns that map to the data, specify how the data is delimited, and provide the location in Amazon S3 for the file(s). 


<img src="img/athena_register_tsv.png" width="60%" align="left">

In [1]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [2]:
ingest_create_athena_table_tsv_passed = False

In [3]:
%store -r ingest_create_athena_db_passed

In [4]:
try:
    ingest_create_athena_db_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [5]:
print(ingest_create_athena_db_passed)

True


In [6]:
if not ingest_create_athena_db_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [7]:
%store -r s3_private_path_tsv

In [8]:
try:
    s3_private_path_tsv
except NameError:
    print("*****************************************************************************")
    print("[ERROR] PLEASE RE-RUN THE PREVIOUS COPY TSV TO S3 NOTEBOOK ******************")
    print("[ERROR] THIS NOTEBOOK WILL NOT RUN PROPERLY. ********************************")
    print("*****************************************************************************")

In [9]:
print(s3_private_path_tsv)

s3://sagemaker-us-east-1-546928460657/airline-delay-cause/csv


# Import PyAthena

In [10]:
from pyathena import connect

# Create Athena Table from Local CSV File

#### Dataset columns
- `year`: YYYY format
- `month`: MM format (1-12)
- `carrier`: Code assigned by assigned by US DOT to identify a unique airline carrier.  
- `carrier_name`: Unique airline (carrier) is defined as one holding and reporting under the same DOT certificate regardless of its Code, Name, or holding company/corporation.
- `airport`: A three character alpha-numeric code issued by the U.S. Department of Transportation which is the official designation of the airport.
- `airport_name`: a place from which aircraft operate that usually has paved runways and maintenance facilities and often serves as a terminal
- `arr_flights`: Arrival Flights
- `arr_del15`: Arrival Delay Indicator, 15 Minutes or More Arrival delay equals the difference of the actual arrival time minus the scheduled arrival time. A flight is considered on-time when it arrives less than 15 minutes after its published arrival time.
- `carrier_ct`:	Carrier Count for airline cause of delay
- `weather_ct`:	Weather Count for airline cause of delay
- `nas_ct`:	NAS (National Air System) Count for airline cause of delay
- `security_ct`: Security County for airline cause of delay
- `late_aircraft_ct`: Late Aircraft Delay Count for airline cause of delay
- `arr_cancelled`: flight cancelled
- `arr_diverted`: flight diverted
- `arr_delay`: Difference in minutes between scheduled and actual arrival time. Early arrivals show negative numbers.
- `carrier_delay`: Carrier Delay, in Minutes
- `weather_delay`: Weather Delay, in Minutes
- `nas_delay`: National Air System Delay, in Minutes
- `security_delay`: Security Delay, in Minutes
- `late_aircraft_delay`: Late Aircraft Delay, in Minutes



In [11]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [12]:
# Set Athena parameters
database_name = "db_airline_delay_cause"
table_name_tsv = "airline_delay_cause_csv"

In [13]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [35]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         year string,
         month string,
         carrier string,
         carrier_name string,
         airport string,
         airport_name string,
         arr_flights float,
         arr_del15 float,
         carrier_ct float,
         weather_ct float,
         nas_ct float,
         security_ct float,
         late_aircraft_ct float,
         arr_cancelled int,
         arr_diverted int,
         arr_delay int,
         carrier_delay int,
         weather_delay int,
         nas_delay int,
         security_delay int,
         late_aircraft_delay int
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')""".format(
    database_name, table_name_tsv, s3_private_path_tsv
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS db_airline_delay_cause.airline_delay_cause_csv(
         year string,
         month string,
         carrier string,
         carrier_name string,
         airport string,
         airport_name string,
         arr_flights float,
         arr_del15 float,
         carrier_ct float,
         weather_ct float,
         nas_ct float,
         security_ct float,
         late_aircraft_ct float,
         arr_cancelled int,
         arr_diverted int,
         arr_delay int,
         carrier_delay int,
         weather_delay int,
         nas_delay int,
         security_delay int,
         late_aircraft_delay int
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-546928460657/airline-delay-cause/csv'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')


In [36]:
import pandas as pd

pd.read_sql(statement, conn)

/tmp/ipykernel_516/3803073958.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


# Verify The Table Has Been Created Succesfully

In [37]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_516/2201015668.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,airline_delay_cause_csv


In [38]:
if table_name_tsv in df_show.values:
    ingest_create_athena_table_tsv_passed = True

In [39]:
%store ingest_create_athena_table_tsv_passed

Stored 'ingest_create_athena_table_tsv_passed' (bool)


# Run A Sample SQL Query

In [40]:
carrier = "9E"

statement = """SELECT * FROM {}.{}
    WHERE carrier = '{}' LIMIT 100""".format(
    database_name, table_name_tsv, carrier
)

print(statement)

df = pd.read_sql(statement, conn)
df.head(5)

SELECT * FROM db_airline_delay_cause.airline_delay_cause_csv
    WHERE carrier = '9E' LIMIT 100


/tmp/ipykernel_516/1806028933.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2024,9,9E,Endeavor Air Inc.,ABE,"""Allentown/Bethlehem/Easton",None,81.0,7.0,5.61,...,0.84,0.0,0,0,0,243,201,15,27,0
1,2024,9,9E,Endeavor Air Inc.,AEX,"""Alexandria",None,81.0,6.0,4.43,...,1.13,0.0,0,0,0,647,440,0,189,0
2,2024,9,9E,Endeavor Air Inc.,AGS,"""Augusta",None,133.0,12.0,3.49,...,3.84,0.0,2,10,0,2382,1269,594,167,0
3,2024,9,9E,Endeavor Air Inc.,ALB,"""Albany",None,73.0,4.0,0.82,...,0.23,0.0,1,0,0,160,28,15,9,0
4,2024,9,9E,Endeavor Air Inc.,ATL,"""Atlanta",None,2204.0,280.0,61.70,...,88.36,0.0,112,40,4,24383,7840,2297,3690,0


# Load Table into Dataframe to Process with Pandas (Note I'm limiting this to 1000 due to the size of the Data. Do not do this on your homework)

In [42]:
statement = """SELECT * FROM {}.{} LIMIT 1000""".format(
    database_name, table_name_tsv,
)

df = pd.read_sql(statement, conn)

/tmp/ipykernel_516/924531533.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


# Run a Sample Pandas Query

In [43]:
print(df.loc[df['carrier'] == '9E'])
df.head(5)

    year month carrier       carrier_name airport  \
0   2024     9      9E  Endeavor Air Inc.     ABE   
1   2024     9      9E  Endeavor Air Inc.     AEX   
2   2024     9      9E  Endeavor Air Inc.     AGS   
3   2024     9      9E  Endeavor Air Inc.     ALB   
4   2024     9      9E  Endeavor Air Inc.     ATL   
..   ...   ...     ...                ...     ...   
91  2024     9      9E  Endeavor Air Inc.     TVC   
92  2024     9      9E  Endeavor Air Inc.     TYS   
93  2024     9      9E  Endeavor Air Inc.     VLD   
94  2024     9      9E  Endeavor Air Inc.     VPS   
95  2024     9      9E  Endeavor Air Inc.     XNA   

                   airport_name arr_flights  arr_del15  carrier_ct  \
0   "Allentown/Bethlehem/Easton        None       81.0         7.0   
1                   "Alexandria        None       81.0         6.0   
2                      "Augusta        None      133.0        12.0   
3                       "Albany        None       73.0         4.0   
4            

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2024,9,9E,Endeavor Air Inc.,ABE,"""Allentown/Bethlehem/Easton",None,81.0,7.0,5.61,...,0.84,0.0,0.0,0.0,0.0,243.0,201.0,15.0,27.0,0.0
1,2024,9,9E,Endeavor Air Inc.,AEX,"""Alexandria",None,81.0,6.0,4.43,...,1.13,0.0,0.0,0.0,0.0,647.0,440.0,0.0,189.0,0.0
2,2024,9,9E,Endeavor Air Inc.,AGS,"""Augusta",None,133.0,12.0,3.49,...,3.84,0.0,2.0,10.0,0.0,2382.0,1269.0,594.0,167.0,0.0
3,2024,9,9E,Endeavor Air Inc.,ALB,"""Albany",None,73.0,4.0,0.82,...,0.23,0.0,1.0,0.0,0.0,160.0,28.0,15.0,9.0,0.0
4,2024,9,9E,Endeavor Air Inc.,ATL,"""Atlanta",None,2204.0,280.0,61.70,...,88.36,0.0,112.0,40.0,4.0,24383.0,7840.0,2297.0,3690.0,0.0


In [44]:
if not df.empty:
    print("[OK]")
else:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")

[OK]


# Review the New Athena Table in the Glue Catalog

In [45]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#">AWS Glue Catalog</a></b>'.format(
            region
        )
    )
)

/tmp/ipykernel_516/4130537117.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Store Variables for the Next Notebooks

In [46]:
%store

Stored variables and their in-db values:
ingest_create_athena_db_passed                    -> True
ingest_create_athena_table_tsv_passed             -> True
s3_private_path_tsv                               -> 's3://sagemaker-us-east-1-546928460657/airline-del
setup_dependencies_passed                         -> True
setup_s3_bucket_passed                            -> True


# Release Resources

In [47]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}